# Importar librerias

In [1]:
!pip install datasets transformers
!pip install datasets[image]
!pip install tensorflow keras
!pip install optuna

In [1]:
import pandas as pd
import os
import shutil
import random
import torch
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing import image
from IPython.display import Image, display

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import ModelCheckpoint

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot, make_confusion_matrix

2025-05-19 10:20:30.379894: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 10:20:30.389354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747642830.400211 1903920 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747642830.403324 1903920 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747642830.412218 1903920 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

--2025-05-19 10:20:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[185.199.109.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 10246 (10K) [text/plain]
Guardando como: ‘helper_functions.py.17’

helper_functions.py 100%[===================>]  10,01K  --.-KB/s    en 0,001s  

2025-05-19 10:20:31 (18,6 MB/s) - ‘helper_functions.py.17’ guardado [10246/10246]



# Cargar Dataset


In [2]:
# Cargo dataset
data = pd.read_csv("plantasMod.csv", delimiter=';' )

# Para mostrar las primeras filas del DataFrame
print(data.head())


                                     image_name   organ  species_id  \
0  b38cba131753842b02d1145e044ff81bdfea189a.jpg    leaf     1397299   
1  61ad9bbd1d272afab922b03871cc74292f0a0e52.jpg   fruit     1397299   
2  3c1319dad4806a20ee7839eae682a57f2933c359.jpg   fruit     1397299   
3  9de3671e5e1d55d985dd1259631ee12f9245f32e.jpg    bark     1397299   
4  55d664f8ac88de2d5b3100b1095c64eb8f234af1.jpg  flower     1397299   

       obs_id   license partner                       author  altitude  \
0  1009361531  cc-by-sa     NaN           Dr. Kurt Haberkorn       NaN   
1  1012296069  cc-by-sa     NaN                        Lucas  1091.858   
2  1005096474  cc-by-sa     NaN                 Pavlík Lukáš   284.000   
3  1020295117  cc-by-sa     NaN               Sylvain Gaudin  1166.300   
4  1004639148  cc-by-sa    tela  Tela Botanica − Alain Bigou   967.000   

    latitude  longitude  gbif_species_id           species  genus    family  \
0        NaN        NaN        2685484.0  Abies a

# Preparación y estudio del dataset

In [3]:
# Me quedo con las columnas que me interesan
dataset = data[['image_name', 'species']]

# Compruebo que me he quedado con lo que me interesa
print(dataset.head())

# Veo cuantas clases hay
num_species = dataset['species'].nunique()
print(f"Numero de especies: {num_species}")

# Calculo el numero de fotos por especies
cuenta_imagenes = dataset['species'].value_counts()

# veo cual son el maximo y minimo de fotos por especie
max_imagenes = cuenta_imagenes.max()
min_imagenes = cuenta_imagenes.min()

print(f"Maximo de imagenes en especie: {max_imagenes}")
print(f"Minimo de imagenes en especie: {min_imagenes}")


                                     image_name           species
0  b38cba131753842b02d1145e044ff81bdfea189a.jpg  Abies alba Mill.
1  61ad9bbd1d272afab922b03871cc74292f0a0e52.jpg  Abies alba Mill.
2  3c1319dad4806a20ee7839eae682a57f2933c359.jpg  Abies alba Mill.
3  9de3671e5e1d55d985dd1259631ee12f9245f32e.jpg  Abies alba Mill.
4  55d664f8ac88de2d5b3100b1095c64eb8f234af1.jpg  Abies alba Mill.
Numero de especies: 4950
Maximo de imagenes en especie: 50
Minimo de imagenes en especie: 1


# Cargar modelo

In [4]:
# Cargar el modelo y el procesador (necesario para trabajar con imágenes)
modelo_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(229, 229, 3))

I0000 00:00:1747642832.915481 1903920 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1006 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
modelo_base.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 229, 229,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 114, 114,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 114, 114,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 114, 114,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 55, 55,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 55, 55,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 55, 55,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 53, 53,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 53, 53,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 53, 53,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [6]:
# Congelar todo primero
for layer in modelo_base.layers:
    layer.trainable = False

# Luego descongelar las últimas capas
for layer in modelo_base.layers[-20:]:
    layer.trainable = True

model = modelo_base.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dense(1206, activation='relu')(model)
model = tf.keras.layers.Dropout(rate=0.3286)(model)
model = tf.keras.layers.Dense(4950,activation='softmax')(model)
model = tf.keras.models.Model(inputs=modelo_base.input,outputs=model)


model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 229, 229,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 114, 114,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 114, 114,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 114, 114,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 55, 55,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 55, 55,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 55, 55,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 53, 53,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 53, 53,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 53, 53,    │          0 │ batch_normalizat

 Total params: 30,248,528 (115.39 MB)

 Trainable params: 10,381,104 (39.60 MB)

 Non-trainable params: 19,867,424 (75.79 MB)

In [7]:
from tensorflow.keras.losses import CategoricalCrossentropy

loss_fn = CategoricalCrossentropy(label_smoothing=0.1)

optimizer = Adam(learning_rate=0.0005875316434787359)
model.compile(optimizer=optimizer, loss=loss_fn, metrics= ['accuracy'])

checkpoint = ModelCheckpoint("modelos/x.h5", monitor="val_accuracy", save_best_only=True, mode="auto", verbose=1)

# Entrenamiento

preparación del aumento de datos

In [8]:
# aumento de datos de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8,1.2],
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
)
# Aumento de datos para validación
val_datagen = ImageDataGenerator(rescale=1./255)

# Aumento de datos para prueba
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
# Cargar imágenes de entrenamiento
train = train_datagen.flow_from_directory(
    'dividido/train',
    target_size=(229, 229),
    batch_size=32,
    class_mode='categorical'
)

# Cargar imágenes de validación
validation = val_datagen.flow_from_directory(
    'dividido/val',
    target_size=(229, 229),
    batch_size=32,
    class_mode='categorical'
)

# Cargar imágenes de prueba
test = test_datagen.flow_from_directory(
    'dividido/test',
    target_size=(229, 229),
    batch_size=32,
    class_mode='categorical'
)

Found 139417 images belonging to 4950 classes.
Found 39634 images belonging to 4950 classes.
Found 20998 images belonging to 4950 classes.


In [10]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train = train.classes  # ahora ya está disponible

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

import collections
counts = collections.Counter(train.classes)

# Mostrar clases con menos de 10 imágenes
rare_classes = {k: v for k, v in counts.items() if v < 10}
print(f'Clases con pocas imágenes: {rare_classes}')

Clases con pocas imágenes: {18: 3, 35: 4, 45: 1, 52: 4, 54: 7, 91: 2, 93: 1, 94: 1, 98: 7, 100: 4, 105: 4, 110: 2, 115: 1, 116: 4, 117: 1, 119: 1, 122: 1, 123: 2, 127: 1, 131: 3, 132: 4, 133: 4, 138: 2, 143: 7, 144: 3, 145: 6, 146: 2, 148: 5, 150: 9, 168: 1, 170: 9, 174: 1, 194: 2, 195: 3, 200: 7, 231: 4, 250: 3, 277: 1, 280: 1, 315: 1, 317: 7, 322: 1, 326: 1, 332: 7, 335: 6, 337: 7, 340: 5, 346: 9, 350: 4, 356: 2, 362: 1, 368: 1, 375: 8, 378: 3, 388: 1, 392: 9, 396: 4, 398: 4, 400: 7, 403: 8, 413: 9, 424: 1, 427: 2, 428: 1, 429: 7, 430: 7, 431: 1, 432: 7, 433: 5, 437: 1, 438: 7, 441: 7, 442: 9, 443: 1, 444: 1, 447: 1, 465: 4, 496: 4, 516: 1, 527: 3, 528: 8, 529: 1, 540: 4, 543: 1, 547: 6, 561: 8, 564: 1, 570: 1, 572: 4, 580: 1, 596: 1, 597: 8, 602: 1, 603: 3, 618: 7, 622: 9, 623: 1, 626: 1, 647: 1, 650: 1, 652: 9, 654: 8, 655: 1, 656: 1, 657: 7, 660: 5, 661: 4, 662: 1, 666: 7, 667: 4, 674: 8, 695: 6, 709: 6, 716: 4, 717: 1, 722: 1, 724: 4, 742: 8, 746: 9, 765: 4, 776: 4, 786: 9, 789: 

preparación del entrenamiento

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',    
    patience=3,             
    restore_best_weights=True
)

history = model.fit(
          train,
          validation_data=validation,
          verbose=1,
          batch_size=32,
          shuffle=True,
          callbacks=[checkpoint, early_stopping],
          class_weight=class_weights,
          epochs=100)

/home/gambito/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


I0000 00:00:1747642841.691179 1914182 service.cc:152] XLA service 0x7fde28002b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747642841.691194 1914182 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2025-05-19 10:20:41.808344: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747642842.950763 1914182 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-05-19 10:20:43.765539: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9305', 376 bytes spill stores, 376 bytes spill loads

2025-05-19 10:20:44.125080: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_100

   1/4357 ━━━━━━━━━━━━━━━━━━━━ 14:42:30 12s/step - accuracy: 0.0000e+00 - loss: 8.2203

I0000 00:00:1747642850.057205 1914182 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

4357/4357 ━━━━━━━━━━━━━━━━━━━━ 1258s 289ms/step - accuracy: 0.0294 - loss: 7.2013 - val_accuracy: 0.0379 - val_loss: 7.0274
Epoch 26/100
4357/4357 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.0315 - loss: 7.0315
Epoch 26: val_accuracy improved from 0.03792 to 0.04166, saving model to modelos/x.h5


4357/4357 ━━━━━━━━━━━━━━━━━━━━ 1263s 290ms/step - accuracy: 0.0315 - loss: 7.0316 - val_accuracy: 0.0417 - val_loss: 6.9964
Epoch 27/100
4357/4357 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.0337 - loss: 7.0366
Epoch 27: val_accuracy improved from 0.04166 to 0.04456, saving model to modelos/x.h5


4357/4357 ━━━━━━━━━━━━━━━━━━━━ 1258s 289ms/step - accuracy: 0.0337 - loss: 7.0366 - val_accuracy: 0.0446 - val_loss: 6.9479
Epoch 28/100
4357/4357 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.0367 - loss: 6.9669
Epoch 28: val_accuracy did not improve from 0.04456
4357/4357 ━━━━━━━━━━━━━━━━━━━━ 1262s 290ms/step - accuracy: 0.0367 - loss: 6.9669 - val_accuracy: 0.0442 - val_loss: 6.9425
Epoch 29/100
4357/4357 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.0378 - loss: 6.9017
Epoch 29: val_accuracy improved from 0.04456 to 0.04809, saving model to modelos/x.h5


4357/4357 ━━━━━━━━━━━━━━━━━━━━ 1264s 290ms/step - accuracy: 0.0378 - loss: 6.9017 - val_accuracy: 0.0481 - val_loss: 6.8915
Epoch 30/100
4357/4357 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.0424 - loss: 6.8502
Epoch 30: val_accuracy improved from 0.04809 to 0.04965, saving model to modelos/x.h5


4357/4357 ━━━━━━━━━━━━━━━━━━━━ 1264s 290ms/step - accuracy: 0.0424 - loss: 6.8502 - val_accuracy: 0.0497 - val_loss: 6.8816
Epoch 31/100
 802/4357 ━━━━━━━━━━━━━━━━━━━━ 15:04 254ms/step - accuracy: 0.0456 - loss: 6.7635

# Evaluación


In [ ]:
accu = model.evaluate(test)
print("Accuracy: ", accu)

In [ ]:
def plot_loss_curves1(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.ylim(0, 1)  # y-axis from 0 to 1
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.ylim(0, 1)  # y-axis from 0 to 1
  plt.legend();

plot_loss_curves1(history)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Predecir
y_pred_probs = model.predict(test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Obtener etiquetas verdaderas (para ImageDataGenerator)
y_true = test.classes

# Matriz de confusión (solo parte visualizable)
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm[:50, :50], annot=False, fmt='d', cmap='Blues')  # solo primeras 50 clases
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Partial View)')
plt.show()

# Clasification Report con F1-score por clase
report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

# Convertimos a DataFrame para ver como tabla
report_df = pd.DataFrame(report_dict).transpose()

# Mostramos las primeras filas 
print(report_df.head(10))

# Guardar en CSV para análisis
report_df.to_csv("classification_report_f1.csv", index=True)


In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam


# Cargar imágenes de entrenamiento
train = train_datagen.flow_from_directory(
    'prueba/train',
    target_size=(229, 229),
    batch_size=16,
    class_mode='categorical'
)

# Cargar imágenes de validación
validation = val_datagen.flow_from_directory(
    'prueba/val',
    target_size=(229, 229),
    batch_size=16,
    class_mode='categorical'
)



def objective(trial):
    # Hiperparámetros a optimizar
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.7)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    dense_units = trial.suggest_int("dense_units", 512, 2048)

    # Base del modelo
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(229, 229, 3))
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(dense_units, activation='relu')(x) 
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
    predictions = tf.keras.layers.Dense(56, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

    # Congelar la base para acelerar la búsqueda
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print("compilado")

    history = model.fit(
        train,
        validation_data=validation,
        epochs=5,  
        batch_size=16,
        verbose=1  
    )

    val_loss = history.history['val_loss'][-1]
    return val_loss
    
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("Mejores hiperparámetros:", study.best_params)


Found 1752 images belonging to 56 classes.
Found 500 images belonging to 56 classes.


[I 2025-05-14 13:17:52,903] A new study created in memory with name: no-name-8d54a19d-4f83-44b1-9d96-1abd3a4e9dad


compilado
Epoch 1/5


I0000 00:00:1747221477.382904  300744 service.cc:152] XLA service 0x7fcae8002260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747221477.382920  300744 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2025-05-14 13:17:57.477089: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747221478.527633  300744 cuda_dnn.cc:529] Loaded cuDNN version 90501
2025-05-14 13:17:59.271779: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_8402', 292 bytes spill stores, 292 bytes spill loads

2025-05-14 13:17:59.562378: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_840

  3/110 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.0243 - loss: 4.5279      

...

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.0350 - loss: 4.1541

2025-05-14 13:49:42.688363: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2464', 88 bytes spill stores, 88 bytes spill loads

2025-05-14 13:49:42.698933: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2464', 956 bytes spill stores, 956 bytes spill loads

2025-05-14 13:49:46.849017: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2457', 300 bytes spill stores, 300 bytes spill loads

2025-05-14 13:49:47.048013: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2464', 80 bytes spill stores, 80 bytes spill loads

2025-05-14 13:49:47.101471: I extern

110/110 ━━━━━━━━━━━━━━━━━━━━ 32s 232ms/step - accuracy: 0.0352 - loss: 4.1528 - val_accuracy: 0.1120 - val_loss: 3.5926
Epoch 2/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 16s 142ms/step - accuracy: 0.1246 - loss: 3.4986 - val_accuracy: 0.1740 - val_loss: 3.2927
Epoch 3/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 15s 141ms/step - accuracy: 0.1957 - loss: 3.1741 - val_accuracy: 0.2040 - val_loss: 3.0926
Epoch 4/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 16s 141ms/step - accuracy: 0.2460 - loss: 2.9053 - val_accuracy: 0.2400 - val_loss: 2.9452
Epoch 5/5
110/110 ━━━━━━━━━━━━━━━━━━━━ 16s 141ms/step - accuracy: 0.2603 - loss: 2.7475 - val_accuracy: 0.2680 - val_loss: 2.8541


[I 2025-05-14 13:50:50,683] Trial 19 finished with value: 2.8540873527526855 and parameters: {'dropout_rate': 0.3657544586728571, 'learning_rate': 0.00012868301686393442, 'dense_units': 1658}. Best is trial 3 with value: 2.7493762969970703.


Mejores hiperparámetros: {'dropout_rate': 0.32862326208601705, 'learning_rate': 0.0005875316434787359, 'dense_units': 1206}
